In [ ]:
#!pip install ibm-cos-sdk

# IBM COS SDK - Test Notebook

### List bucket files

##### local directory

In [ ]:
import os
from glob import glob

all_local_csv_files = []
PATH = "./"
EXT = "*.csv"

for path, subdir, files in os.walk(PATH):
    for file in glob(os.path.join(path, EXT)):
        # skip file with columns only csv
        filesize=os.stat(file).st_size
        if (filesize>8200):
            all_local_csv_files.append(file)
            
#print(all_csv_files)
print(len(all_local_csv_files))
for fi in all_local_csv_files:
    print (fi)

##### cos bucket directory

In [ ]:
from ibm_botocore.client import Config, ClientError
import ibm_boto3
import pandas as pd
import io

pd.set_option('display.max_columns', None)
#pd.set_option('max_rows', None)

MY_ACCESS_KEY_ID=""
MY_SECRET_ACCESS_KEY=""
COS_ENDPOINT="https://s3.us-east.cloud-object-storage.appdomain.cloud"
COS_BUCKET_PUBLIC_DATASET=""

In [ ]:
"""List Buckets"""
s3_client = ibm_boto3.client("s3", aws_access_key_id=MY_ACCESS_KEY_ID,  aws_secret_access_key=MY_SECRET_ACCESS_KEY, endpoint_url=COS_ENDPOINT)

for bucket in s3_client.list_buckets()['Buckets']:
    print(bucket['Name'])
    if (bucket['Name']=="my-analytic-zone"):
        print(bucket)

In [ ]:
"""List Bucket Objects"""
all_bucket_csv_files = []
s3_resource = ibm_boto3.resource("s3", 
        aws_access_key_id=MY_ACCESS_KEY_ID, 
        aws_secret_access_key=MY_SECRET_ACCESS_KEY, 
        endpoint_url=COS_ENDPOINT)

#public_bucket = s3_resource.Bucket(name='awb-ddeid-dataset-public')
public_bucket = s3_resource.Bucket(name=COS_BUCKET_PUBLIC_DATASET)

for bucket_object in public_bucket.objects.all():
     if bucket_object.key.endswith('.csv'):
            #filter meta data
            if "metadata" not in bucket_object.key: 
                all_bucket_csv_files.append(bucket_object.key)

In [ ]:
print(len(all_bucket_csv_files))
all_bucket_csv_files

In [ ]:
def get_dataframe_from_cos(s3_resource,bucket_name, key):
    obj = s3_resource.Object(bucket_name=bucket_name, key=key).get()
    return pd.read_csv(io.BytesIO(obj['Body'].read()), error_bad_lines=False, index_col=False, dtype='unicode',low_memory=False)

In [ ]:
first_csv_file=all_bucket_csv_files[0]

In [ ]:
# # Read COS file content into Panda DataFrame
# pd3 = get_dataframe_from_cos(s3_resource,bucket_name=COS_BUCKET_PUBLIC_DATASET, key=first_csv_file)

In [ ]:
# pd3.head(5)